In [ ]:
import os
import tiktoken
import openai
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

openai.api_type = "azure"
openai.base_url = os.environ.get("OPENAI_API_BASE")
openai.api_key = os.environ.get("OPENAI_API_KEY")
openai.api_version = "2023-05-15"


CHAT_MODEL =  os.environ.get("OPENAI_CHAT_DEPLOYMENT_NAME") 
encoding = tiktoken.get_encoding('cl100k_base')

### Set the Temperature

In [ ]:
import os
from openai import AzureOpenAI

client = AzureOpenAI(
  api_key = os.getenv("OPENAI_API_KEY"),  
  api_version = "2023-05-15",
  azure_endpoint = os.getenv("OPENAI_API_BASE")
)

for x in range(10):
    response = client.chat.completions.create(
        temperature=0.1, # this is the degree of randomness of the model's output. A higher temperature (>1) makes the model more random, diverse, and creative, leading to unexpected or imaginative responses. A lower temperature (<1) makes the model more predictable and deterministic, meaning it will stick closer to common responses.
        model="drn-chat-dev2", # This is currently gpt-35-turbo
        messages=[
            {"role": "system", "content": "Assistant provides single word answers."},
            {"role": "user", "content": "What would be a programmers favorite Food? Give me the single food name only."}
        ]
    )
    print(response.choices[0].message.content)

In [ ]:
def get_completion(prompt, model=CHAT_MODEL):
    response = client.chat.completions.create(
        temperature=0, # this is the degree of randomness of the model's output
        model="drn-chat-dev2", # This is currently gpt-35-turbo
        messages=[
            {"role": "system", "content": "Assistant gives consistent answers."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

### Be specific!

In [ ]:

sampletext = "Prompt engineering is a process in machine learning, specifically in Natural Language Processing (NLP), where prompts are designed to guide the model to provide the desired output. This technique is particularly used in transformer-based models like GPT-3. The quality of the prompt can significantly affect the model's performance. It involves crafting questions or statements in a way that helps the model understand the context and give relevant responses. It can be considered as a form of 'soft coding' to instruct an AI model."

prompt = f"""
Summarize the text between two lines of triple dashes into a single sentence.
---
{sampletext}
---
"""
response = get_completion(prompt)
print(response)

⚠️The above method prevents Prompt Injection as well! As long as they dont know your delimiter strategy.

### Self check!

In [ ]:

sampletext = "1+1*2+3? equals what?"

prompt = f"""
Summarize the text between two lines of triple dashes into a single sentence. Say "No Text Provided" if there is no text between the triple dashes.
---
{sampletext}
---
"""
response = get_completion(prompt)
print(response)

### Few Shot Prompting

References:   
https://arxiv.org/pdf/2302.13971.pdf   
https://arxiv.org/abs/2001.08361   
https://arxiv.org/abs/2005.14165   

In [ ]:
prompt = f"""
Answer questions in a consistent style following the examples provided. 

<q>: How can I learn to be a good programmer?
<a>: In the realm of codes, where logic dances and sways,
A quest to be a good programmer, you wish to pave your way.
Fear not, for I shall guide you, in an enchanting lyrical play.
Dive deep into the ocean of languages, where C++, Java, and Python lay,
Choose your vessel wisely, let passion lead your way.

<q>: How do I maximize my credit card rewards?
"""

response = get_completion(prompt)
print(response)

### Time to think! / Chain-of-Thought (CoT) Prompting

Reference: https://arxiv.org/abs/2201.11903

In [ ]:
prompt = f"""
1+1+(2*2^2)=?
"""

response = get_completion(prompt)
print(response)

# Correct Answer = 10

In [ ]:
calculation = f"""
1+1+(2*2^2)=?
"""

prompt = f"""
Follow the steps below and provide your answer for each step
1. Calculate the section between parantheses
2. replace the paranteses with the result
3. calculate the result

---
{calculation}
---
"""

response = get_completion(prompt)
print(response)

# Correct Answer = 10

Let the model go step by step.

In [ ]:
prompt = f"""
Give me the steps I need to follow to calculate the following?
1+1+(2*2^2)=?
"""

response = get_completion(prompt)
print(response)

### Hallucinations! Living in the dream world!

Reference:  https://www.linkedin.com/pulse/chatgpt-35-example-blatant-hallucination-earl-fong-pbsnc/

In [ ]:
prompt = f"""
Which author of a book on software development said "don't flip the bozo bit"
"""

response = get_completion(prompt)
print(response)


Correct Answer = Jim McCarthy’s book [Dynamics of Software Development](https://amzn.to/4bAUVYd)  
_“don’t flip the bozo bit”_ is McCarthy’s advice not to permanently label any team member as a hopeless idiot.  
Ref: https://medium.com/@mike.besso/do-not-flip-the-bozo-bit-b06fb9395845  
The authors of the [Pragmatic Programmer](https://amzn.to/3XzAaGy) are Andy Hunt and Dave Thomas  